In [ ]:
import json
import re
import statistics

from pyannote.metrics.diarization import DiarizationErrorRate
from pyannote.core import Annotation, Segment

In [ ]:
# Calculates accurateness of Whisper ASR

# Normalizes the text by removing punctuation and converting to lowercase.
def normalize_text(text):
    text = re.sub(r"[^\w\s]", "", text)
    text = text.lower()
    return text


# Calculates the Word Error Rate (WER).
def wer(reference, hypothesis):
    ref_words = normalize_text(reference).split()
    hyp_words = normalize_text(hypothesis).split()

    # Initializes matrix
    d = [[0] * (len(hyp_words) + 1) for _ in range(len(ref_words) + 1)]

    # Fills the first row and column of the matrix
    for i in range(len(ref_words) + 1):
        d[i][0] = i
    for j in range(len(hyp_words) + 1):
        d[0][j] = j

    # Populates matrix
    for i in range(1, len(ref_words) + 1):
        for j in range(1, len(hyp_words) + 1):
            if ref_words[i - 1] == hyp_words[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    # The WER is the number of edits divided by the number of words in the reference
    wer_value = d[len(ref_words)][len(hyp_words)] / len(ref_words)

    return wer_value

In [ ]:
wer_rates = []
wer_rates_covid = []


In [ ]:
# ref is whisper
# hyp is manual

# Haarlem - 2016 - 246556.mp4 - 36:07
ref = "Als er iemand stemming vraagt overigens. Als niemand stemming vraagt. Ja, wel vanuit de SP wordt dat gevraagd. Wie in de regio? Dat is dat gevraagd. Wie is voor dit beleidskader? Nou had ik beter eerst kunnen vragen wie tegen was. 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20. Ik zie de SP die stemt niet voor. Even kijken. Ja, de rest wel. Dus met stemmen van de SP tegen is het aangenomen. Dan gaan wij naar agenda punt 9. Dat is het on-eigelijk gebruik van gemeentegrond. Ik heb begrepen dat er wat punten zijn aangepast na de behandeling in de commissie. Maar dat de wethouder nog even een toevoeging heeft op die aanpassingen. De heer Bottigen. Meneer de voorzitter. Er is in de stukken een fout geslopen in het collegebesluit. Wat als raadsbesluit staat. Daar wordt wel gesproken over dat het gaat over 2,2 miljoen verdeeld over 10 jaar. Maar dat dat per jaar wordt vastgesteld. Maar in het raadsbesluit is dat verkeerd opgenomen. Dus de tekst van het stuk van het raadsbesluit. Het is zo dat eigenlijk daarmee die motie die is ingediend op dit vlak. Of het abonnement. Waarbij staat dat er gedurende het bedrag van de kadernoten van vier jaar. En dat dan per jaar wordt gekeken of dat bedrag ingezet kan worden. Dat is ook de formulering zoals het college die heeft gedaan. Alleen die is in het raadsvoorstel onvoldoende of niet verwerkt. Dus we zijn het eens met elkaar. Waarbij wel de kanttekening overigens dat het geen zin heeft om bijvoorbeeld volgend jaar al te zeggen. Van we hebben het een jaartje geprobeerd en het werkt niet"
hyp = "Als er iemand stemming vraagt overigens. Als niemand stemming vraagt. Ja Wel vanuit de SP wordt dat gevraagd. Wie in de regio? Wie is voor dit beleidskader? Nou had ik beter eerst kunnen vragen wie tegen was geloof ik. 5 6 7 8 9 10 11 12 13 14 15 16 17, 18, 19, 20. Ik zie de SP die stemt niet voor, even kijken ja de rest wel dus dat is allen met stemmen van de SP tegen is het aangenomen. Dan gaan wij naar agenda punt 9. Dat is het oneigenlijk gebruik van gemeentegrond. ik heb begrepen dat er wat punten zijn aangepast na de behandleing in de commisie maar dat de wethouder nog even een toevoeging heeft op die aapassingen. de heer bottigen. Meneer de voorzitter er is in de stukken een fout geslopen in het collegebesluit wat als raadsbesluit staat er wordt wel gesproken over dat het gaat over 2,2 miljoen verdeeld over 10 jaar maar dat dat per jaar wordt vastgesteld maar in het raadsbesluit is dat verkeerd opgenomen. Dus de tekst van het stuk van het raadsbesluit het is zo dat eigenlijk daarmee die motie die is ingediend op dit vlak of het abonnement waarbij staat dat er gedurende het bedrag nu van de kader noten van de vier jaar en dat dan per jaar wordt gekeken of dat bedrag ingezet kan worden, dat is ook de zeg maar de formulering zoals het college die heeft gedaan. Alleen die is in het raadsvoorstel onvoldoende of niet verwerkt. Dus we zijn het eens met elkaar. Waarbij wel de kanttekening overigens dat het geen zin heeft  om bijvoorbeeld volgend jaar al te zeggen van we hebben het een jaartje geprobeerd en het werkt niet."
error_rate = wer(ref, hyp)
wer_rates.append(error_rate)
print(f"WER: {error_rate:.2f}")

# Haarlem - 2019 - 622012.mp4 - 1:46:45
ref = "Dank u wel. De heer Visser, Christen Unie. Dank u wel, voorzitter. Wat betreft de oude stukken die opnieuw geheim verklaaren kunnen wij akkoord gaan omdat het college zegt daar doen we een deadline op. Zo snel mogelijk in het najaar gaan we er opnieuw over hebben. Wat betreft het PWC-rapport is niet vanaf vandaag het openbaar met uitzondering van de weggelakte delen. Dus dat is ook mooi. Wat betreft dateterrein ben ik toch een beetje benieuwd naar de reactie van het college. En ik heb inderdaad ook het gevoel dat we vanavond toch al op het zuid gaan komen. Dus waarom zouden we dat nu nog geheim verklaaren? Ik ben er niet van overtuigd wat daar het argument voor is. Dus ik zou daar eigenlijk een kort antwoord van het college op willen. Het is tot en met een ding niet akkoord. Al was het maar voor de symboolwerking. Maar bijlagen zeven bij de gedemd oudergeracht de uitgiftetekening. Ik heb net al gezegd ik vind het compleet een nonsens om zo'n tekening geheim te verklaren. Het college mag me wel eens uitleggen waarom dat geheim zou moeten zijn. Dank u wel. De heer van de Raad. Ja, voorzitter. Wij gaan niet akkoord met stukken die binnen 24 uur zijn ingevoerd. Dat hebben we bij het seniorenconvental aangegeven dat we daar niet meer van gediend zijn. Voor de rest heeft u gevraagd om alles in een keer te behandelen. U heeft ook gezegd alles komt terug. Dus u behandelt alles in een keer en het feit dat het terugkomt vinden wij dat niks geheim verklaard hoeft te worden. Dank u."
hyp = "Dank u wel. De heer visser, christen unie. Dank u wel voorzitter, wat betreft de oude stukken die opnieuw geheim verklaren kunnen wij akkoord gaan omdat het college zegt daar doen we een deadline op. zo snel mogelijk in het najaar gaan we het er opnieuw over hebben. wat betreft het pwcrapport is in ieder geval vanaf vandaag openbaar met uitzondering van de weggelakte delen. Dus dat is ook mooi. Wat betreft Dateterrein ben ik toch een beetje benieuwd naar de reactie van het college. En ik heb inderdaad ook het gevoel dat we vanavond toch al een besluit gaan komen dus waarom zouden we dat nu nog geheim verklaren? Ik ben er niet van overtuigd wat daar het argument van is. Dus ik zou daar eigenlijk een kort antwoord van het college op willen. En tenslotte ga ik met een ding niet akkoord, al was het maar voor de symboolwerking. Maar bijlage zeven bij de gedemde ouderdrachte uitgiftetekening. Ik heb het net al gezegd, ik vind het complete onzin om zo'n tekening geheim te verklaren. Het college mag me wel eens uitleggen waarom dat geheim zou moeten zijn. Dank u wel. De heer van de raad. Ja voorzitter we gaan niet akkoord met stukken die binnen 24 uur zijn ingevoerd dat hebben we bij het seniorenconferent al aangegeven dat we daar niet meer van gediend zijn. En voor de rest heeft u gevraaagd om alles in een keer te behandelen en u heeft ook gezegd alles komt terug, dus ik behandel alles in een keer, en het feit dat het terug komt vinden wij dat niks geheim verklaard hoeft te worden dank u."
error_rate = wer(ref, hyp)
wer_rates.append(error_rate)
print(f"WER: {error_rate:.2f}")

# Haarlem - 2024 - 1159903.mp4 - 8:10
ref = "Maar dat heeft niks te maken op zich met het referendum. Dan kan het college bevestigen dat er altijd bezwaar en beroep mogelijk is. Dat klopt niet, want in een aantal gevallen is er helemaal geen verkeersbesluit voor nodig. Iets wat niet legaal is, als je dat op een gegeven moment gaat niet meer gedoogd, dan neem je geen nieuw verkeersbesluit, want het was al illegaal. Dat moet je wel heel duidelijk markeren, want we gaan daar nu op een andere manier mee om. Ik heb al gezegd dat gebeurt alleen als er voldoende parkeerplaatsen beschikbaar zijn om de mensen met een vergunning te laten parkeren. In die situaties waarbij gelegaliseerd parkeren wordt opgeheven, daar is wel een parkeerbesluit voor en dat zal natuurlijk de gewone procedure doorlopen. Het opheffen van parkeerplaatsen, daarvoor geldt ook het reguliere participatie- en inspraakproject als dat zou gebeuren. Dan de vraag over de website. We proberen te voorkomen dat je de website vol stopt met informatie, maar nu dit zozeer een issue is"
hyp = "Maar dat heeft niks te maken op zich, met het referendum. Dan, kan het college bevestigen dat er altijd bezwaer en beroep mogelijk is. Dat klopt niet want in een aantal gevallen is er helemaal geen verkeersbesluit voor nodig. Iets wat niet legaal is, als je dat op een gegeven moment niet meer gedoogd dan neem je geen nieuw verkeersbesluit want het was al illegaal. Dat moet je natuurlijk wel heel duidelijk markeren van we gaan daar nu op een andere manier mee om. ik heb al gezegd dat gebeurt alleen als daar ook de ruimte voor is, als er dus voldoende verkeersplaatsen beschikbaar zijn om de mensen met een vergunning ook te laten parkeren. in die situaties waarbij gelegaliseerd parkeren wordt opgeheven daar is wel een parkeerbesluit voor en dat zal natuurlijk de gewone procedure doorlopen en het opheffen van parkeerplaatsen daarvoor geldt ook het reguliere participatie- en inspraak project als dat zou gebeuren. dan de vraag over de website we proberen te voorkomen dat je de website vol stopt met informatie maar nu dit zozeer een issue is "
error_rate = wer(ref, hyp)
wer_rates.append(error_rate)
print(f"WER: {error_rate:.2f}")

# Barendrecht - 2024 - 1203463.mp4 - 49:36
ref = "Dat is op hun conto volgens de reactie van de wethouder die we vandaag ontvingen, waarbij hij vooral aangeeft onderbouwing te missen en geen aanleiding ziet tot aanpassen. Om het verhaal van beide kanten scherp te krijgen hebben we vandaag nog even gebeld met stad, kraf om deze reactie te toetsen en zij geven aan dat ze meerdere malen met de ambtelijke organisatie gesproken hebben, herkennen het uitblijven van gegevens niet en baseerde hun uitspraak op berekeningen vanuit andere projecten die ze hebben gedaan. Hoe verklaart de wethouder deze communicatie van langs elkaar in plaats van met elkaar? Het komt ons over als een welis nietes discussie, waar we verschillende partijen zien met een mening die niet overeenkomt, wat het lastig maakt voor ons om in te schatten wat nu de waarheid is. Wat in ieder geval duidelijk is voor ons is dat de provincie en stad kraft zich niet kunnen vinden in de voorliggende normen. Inhoudelijk, wil ik eigenlijk nog een keer een standpunt delen wat we over tijdens de orientatie, orienteerende verhalen hebben gedeeld, is dat wij vooral willen pleiten voor impact normen in plaats van input normen. Daarmee bedoelen we impact normen die iets zeggen over de impact op de omgeving, zoals geluidsnormen, slagschaduw enzovoort in plaats van input normen, zoals tiphoogte en afstanden. Dat zegt hoogheid iets over de beeldkwaliteit, maar wordt nu vooral gekoppeld aan geluid en dat is volgens mij een onlogische koppeling. Geluidsnormen moeten we gewoon verwoorden in geluidsnormen. Op die manier geven we ruimte aan de markt om de optimale oplossing te genereren, ook ter gunste van die omgeving. De provincie onderstreept het ook, hoe ze de wethouder dat, impact normen in plaats van input normen. Tot slot beschrijft het voorstel ook een stukje over monitoring en evaluatie, uitgaande dat de ontwikkeling mogelijk is. Stel nu dat de komende tijd toch mogelijk of toch blijkt dat de ontwikkeling niet mogelijk is. Op welke wijze wordt dat geevalueerd en welke vervolgacties worden dan getroffen. Tot zover, dank u wel. Voorzitter. Meneer Bol van Echt-Van Baarneet. Ik heb nu dit keurig even de bijdrage van meneer Koster van GroenLinks afgewacht, maar wat mij opviel u nu betogen is dat u aangeeft, ja ik heb vanmiddag even met de commerciele partij StadKraft gebeld en ze hebben u kennelijk verteld dat, nou wat ons college vertelt, dat dat niet klopt. U knikt ja voor mensen die mij alleen maar horen en u niet zien. Ja ik vind dat toch wel bijzonder en u noemt daar ook het woord vertrouwen in. Dus mijn vraag aan u is, vertrouwt u dan niet dat wat het college ons vanmiddag in een brief heeft geschreven dat dat klopt? En de tweede vraag eigenlijk is, ziet u dan niet in dat StadKraft, wat ik al zei een commerciele partij, dat die ook een bepaald belang heeft om te zorgen dat die winterpunis zo hoog en in aantal ook zoveel mogelijk daar komt. Ziet u ook niet in dat dat misschien ook wellicht gekleurd dan is hetgene wat ze tegen u gezegd hebben?"
hyp = "Dat is op hun conto volgens de reactie van de wethouder die we vandaag ontvingen waarbij hij vooral aangeeft onderbouwing te missen en geen aanleiding ziet tot aanpassen. Nou om het verhaal van beide kanten toch scherp te krijgen hebben we vandaag nog even gebeld met stadskraft om deze reactie te toetsen en zij geven aan dat ze meerdere malen met de ambtelijke organisatie gesproken hebben herkennen het uitblijven van gegevens niet en baseerde hun uitspraak op berekeningen vanuit andere projecten die ze hebben gedaan. Hoe verklaart de wethouder deze communicatie van langs elkaar in plaats van met elkaar? Het komt ons over als een weles nietes discussie waar we verschillende partijen zien met een mening die niet overeenkomt. Wat het lastig maakt voor ons om in te schatten wat nu de waarheid is. Wat in ieder geval duidelijk is voor ons is dat de provincie en stadskraft zich niet kunnen vinden in de voorliggende normen. Inhoudelijk wil ik eigenlijk nog een keer een standpunt delen wat we over tijdens de orientatie orienteerende vergadering hebben gedeeld, is dat wij vooral willen pleiten voor impact normen in plaats van input normen. Daarmee bedoelen we impact normen die iets zeggen over de impact op de omgeving zoals geluidsnormen, slagschaduw enzovoort in plaats van input normen zoals tiphoogte en afstanden. Dat zegt hooguit iets over de beeldkwaliteit maar wordt nu vooral gekoppeld aan geluid en dat is volgens mij een onlogische koppeling, geluidsnormen moeten we gewoon verwoorden in geluidsnormen. Op die manier geven we ruimte aan de markt om de optimale oplossing te genereren, ook ter gunste van de omgeving. De provincie onderstreept het ook. Hoe ze de wethouder dat? impact normen in plaats van input normen. Tot slot schrijft, beschrijft het voorstel ook een stukje over monitoring en evaluatie, uitgaande dat de ontwikkeling mogelijk is. Stel nu dat de komende tijd toch mogelijk, of toch blijkt dat de ontwikkeling niet mogelijk is. Op welke wijze wordt dat geevalueerd en welke vervolgacties worden dan getroffen? Tot zover, dank u wel. Voorzitter. Meneer Bol van Echt Van Baarnet. Ik heb nu dit keurig even de bijdrage van meneer Koster van GroenLinks afgewacht, maar wat mij opviel in uwbetoog is dat u aangeeft van ja ik heb vanmiddag even met de commerciele partij StadsKraft gebeld en ze hebben u kennelijk verteld dat, nou wat ons college vertelt, dat dat niet klopt, als ik dat zo mag u knikt ja voor mensen die mij alleen maar horen en u niet zien. Ja ik vind dat toch wel bijzonder en u noemt daar ook het woord vertrouwen in. Dus mijn vraag aan u is, vertrouwt u dan niet dat wat het college ons vanmiddag in een brief heeft geschreven dat dat klopt? En de tweede vraag eigenlijk is, ziet u dan niet in dat StadsKraft, wat ik al zei een commerciele partij, dat die ook een bepaald belang heeft om te zorgen dat die winterpuzzel hoog en in aantal ook zoveel mogelijk daar komt. Ziet u ook niet in dat dat misschien ook wellicht gekleurd dan is hetgene wat ze tegen u gezegd hebben?"
error_rate = wer(ref, hyp)
wer_rates.append(error_rate)
print(f"WER: {error_rate:.2f}")

# Hoekschewaard - 2023 - 1068492.mp4 - 1:29:25
ref = "dus het lijkt me goed om daar schriftelijk op terug te komen, om ook nog weleens aan te geven wat we er allemaal aan doen. Oke, maar misschien kan dat ook meegenomen worden naar de volgende week, of die kan voor volgende week dan voor de sluitvorming? Technische vragen hoeven ook niet langs het college, dus die kunnen heel snel, die kunnen extra snel. Ja, oke, goed. Ik heb verder, ik moet vragen, de heer Rayen. Ja, dank u wel. Ik vroeg me even af hoe de wethouder aankijkt tegen de uitvoerbaarheid, want er zijn 900 huishoudens om en nabij in de categorie 120 tot 130 procent, van sociaal minimum. Potentieel, misschien gaan het niet iedereen bijzonder bijzonder aanvragen, maar toch zeker wel een aanzienlijk deel. En hoe ziet dat er qua uitvoering uit? En moeten dan niet gewoon geld bij in de uitvoering? En mist dat niet dan in het voorstel? Dank u wel. De weghouder. Voorzitter, de suggestie of de opmerking die de heer Rayen maakt van zeker wel een aanzienlijk deel, dat is nog maar de vraag. Als je kijkt naar waar die energie toetslag, als je kijkt wat de energiekosten op dit moment zijn voor huishouden, het is in opzichte van de periode daarvoor, dan is het nog maar de vraag of heel veel mensen een aanvraag doen als het gaat over de kosten van energie. Daarnaast heeft het natuurlijk ook nog een vermogenstoets, die zit er bij die 120 procent niet bij. Dus op het moment dat je een miljoen euro op de bank hebt, maar 120 procent van het minimum krijgt, krijg je toch wel de energietoetslag, want mij gaat er dan maar vanuit dat als je weinig inkomsten hebt, ook geen vermogen hebt. Dat staat ook zo in het voorstel, dat staat ook zo in de wet genoemd. En bij die 130 procent wordt er dan wel gekeken op de vermogen. Wij denken dat die aanpak op zich binnen de huidige formatie, plus die mogelijke inhuur als dat nodig is, binnen die acht weken te realiseren is. Dat is eigenlijk de vraag die je gesteld hebt. De heer A.M. tevreden? Ja. Mevrouw Verbrug? Ja, bij de vermogenstoets kijken ze ook altijd om mensen een auto hebben. En veel mensen hebben een auto gewoon nodig om naar de werk te komen. En als mensen dan toch die energietoetslag aanvragen en ze hebben een auto, moeten ze die dan eerst verkopen zodat ze een hele oude terug kunnen kopen, voordat ze uberhaupt een vergoeding kunnen krijgen? Ik weet niet of we de weghouden die kan antwoorden, anders moeten we een technisch schriftelijk afdoen. Ja, dit gaat erg, de techniek, en het lijkt wel goed om daar gewoon een goed onderbouwtechnisch antwoord op te geven. Oke. Dan kunnen we voorzorgden dat we daarvoor, bij de besluit van de vergadering, het antwoord hebben daarop. En dat geldt ook nog even terug, want we hebben de vraag van meneer Raaijer. Goed, dan kijk ik verder, heer de Koning. Ja, dank u wel voorzitter. Ik had natuurlijk niet kunnen vermoeden dat ik de vergelijking met een kruidenuur maakte, dat dat eerder als een geuzennaam werd opgepakt als de wethoud. Dus ik moet nu eigenlijk iets anders verzinnen. Nee, laten we het eventjes over het andere aspect. Ik had het over een man, een man, een woord, een woord, en de kruidenuursgeest daar tegenover. En misschien is er niks te naderder van de kruidenuurs."
hyp = "dus het lijkt me goed om daar schriftelijk op terug te komen om ook nog weereens aan te geven wat we er allemaal aan doen. Oke maar misschien kan dat ook meegenomen worden naar de volgende week, of die kan voor volgende week dan voor de besluitvorming? Technische vragen hoeven ook niet langs het college, dus die kunnen heel snel, die kunnen extra snel. Ja oke goed. Ik heb verder, ik moet vragen, de heer Raaijer. Ja, dank u wel. Ik vroeg me even af hoe de wethouder aankijkt tegen de uitvoerbaarheid want er zijn 900 huishoudens om en nabij in de categorie 120 tot 130 procent, van sociaal minimum. Potentieel, nou ja misschien gaat niet iedereen bijzonder bijstand aanvragen, maar toch zeker wel een aanzienlijk deel. En hoe ziet dat er qua uitvoering uit? En moeten dan niet gewoon geld bij in de uitvoering? En mist dat dan niet in het voorstel? Dank u wel. De wethouder. Voorzitter, de suggestie of de opmerking die de heer Raaijer maakt van zeker wel een aanzienlijk deel, dat is nog maar de vraag. Als je kijkt naar waar die energie toeslag, als je kijkt wat de energiekosten op dit moment zijn voor huishoudens, ten opzichte van de periode daarvoor, dan is het nog maar de vraag of heel veel mensen een aanvraag doen als het gaat over de kosten van energie. Daarnaast heeft het natuurlijk ook nog een vermogenstoets, die zit er bij die 120 procent niet bij. Dus op het moment dat je een miljoen euro op de bank hebt, maar 120 procent van het minimum krijgt, krijg je toch wel de energietoetslag, want men gaat er dan maar vanuit dat als je weinig inkomsten hebt, ook geen vermogen hebt. Dat staat ook zo in het voorstel, dat staat ook zo in de wet genoemd. En bij die 130 procent wordt er dan wel gekeken op de vermogen. Wij denken dat die aanpak op zich binnen de huidige formatie, plus die mogelijke inhuur als dat nodig is, binnen die acht weken te realiseren is. Dat is eigenlijk de vraag die u gesteld hebt. De heer Raaijer tevreden? Ja. Mevrouw de brug? Ja, bij de vermogenstoets kijken ze ook altijd om mensen een auto hebben. En veel mensen hebben een auto gewoon nodig om naar hun werk te komen. En als mensen dan toch die energietoeslag aanvragen en ze hebben een auto, moeten ze die dan eerst verkopen zodat ze een hele oude terug kunnen kopen, voordat ze uberhaupt een vergoeding kunnen krijgen? Ik weet niet of we de wethouder dit kan antwoorden, anders moeten we het technisch schriftelijk afdoen. Ja, dit gaat erg, de techniek, en het lijkt wel goed om daar gewoon een goed onderbouw technisch antwoord op te geven. Oke dan kunnen we ervoor zorgen dat we daarvoor, bij de besluit van de vergadering, het antwoord hebben daarop. En dat geldt ook nog even terug, want we hebben de vraag van meneer Raaijer. Goed, dan kijk ik verder, de heer de Koning. Ja, dank u wel voorzitter ja ik had natuurlijk niet kunnen vermoeden dat ik de vergelijking met een kruidenier maakte dat dat eerder als een geuzennaam werd opgepakt als de wethouder. Dus ik moet nu eigenlijk iets anders verzinnen maar Nee, laten we het eventjes over het andere aspect. Ik had het over een man een man een woord een woord, en de kruideniersgeest daar tegenover. En misschien is er niks te naderder van de kruideniers."
error_rate = wer(ref, hyp)
wer_rates.append(error_rate)
print(f"WER: {error_rate:.2f}")

print()

# Covid zoom measurements

# Hoekschewaard - 2021 - 831229.mp4 - 44:51
ref = "meneer de voorzitter, omdat wat ons betreft niet rechtsgeldig is wat wij nu vaststellen. Dank u wel. Heeft u nog wel behoefte aan stemmen mevrouw van Hulst? Nou ja, we kunnen hier dus dan niet mee instemmen, want dan stemmen we met iets in wat niet conform de wet- en regelgeving is. Dat lijkt me niet juist meneer de voorzitter. Dank u wel, dat mag u vinden. Wij gaan de stemming openen. Stemming is geopend, gaat u gaan. 37 stemmen, dames en heren. 34 voor, 3 tegen. En daarmee is het voorstel aan vaart. Dan gaan wij naar agenda.14, coronacompensatie. Geen stemverklaringen, iemand behoefte aan stemming. Nee, dan is het voorstel bij deze aangenomen. Dan gaan wij naar agenda.15 en dat is, ho! Ik heb het voorstel aan de hand. Ik heb het voorstel aan de hand. Ik heb het voorstel aan de hand. En dan gaan we naar agenda.15 en dat is, ho! Het glipt een papiertje uit mijn vingers. Het ambachtcentrum Samen Circulair. Ik zie eens kijken, tisten, stemverklaring van CDA, constructief Hoekswaard en de SGP en we beginnen bij constructief Hoekswaard. Meneer de Jong. Dank u wel, voorzitter. We hebben 3 zorgpunten die ik in deze stemverklaring noem. De 1e zorgpunt is dat de wethouder geen antwoord kan geven op de vraag of dit project zou stoppen als er een renteloos lening gegeven zou worden. In plaats van een aanwezigheid voor de aanwezigheid voor de aanwezigheid voor de aanwezigheid voor de aanwezigheid voor de aanwezigheid voor de aanwezigheid voor de aanwezigheid voor de aanwezigheid voor de aanwezigheid voor de aanwezigheid voor de aanwezigheid voor de aanwezigheid voor de aanwezigheid voor de aanwezigheid voor de Het is een project waar veel overheidssubsidie naartoe gaat en wat zwaar steunt op zorgbudgetten. Er werken kwetsbare mensen die geen enkele vorm van garantie hebben op een gelijkwaardig werktraject als dit plan mocht mislukken. Ook is er geen concreet plan om kringloopwinkels en repaircaf\u00e9s in andere dorpen te faciliteren of aan te laten haken bij het initiatief. De positionering en de bijdrage aan circulaire economie blijft voor het CDA onduidelijk. Wij vinden dat het college ervoor had moeten zorgen dat wij als raad een goede business case voorgelegd hadden gekregen. Toch zullen wij vanavond voor de garantstelling stemmen omdat wij het idee van een pilot voor circulaire economie wel zien zitten. Wij verwachten dat van het college dat zij een vinger aan de pols houdt of er goede stappen worden genomen om business partners aan te haken en fondswerving rond te maken. Dank u wel voorzitter. Dank u wel mevrouw Voorthoerden tot slot de SGP. Dank u wel voorzitter. De SGP kan instellen met dit voorstel omdat het naast het duurzaamheidsaspect ook het verbinden van het dorpskernens centraal staat en een mogelijkheid biedt voor werkervaring voor mensen met een afstand tot de arbeidsmarkt. Wel hebben we onze zorgen of willen we onze zorgen uit over de waarboring van de continu\u00efteit We zien graag dat er toch een concrete inspanningsverplichting komt als die garantstelling wordt gegeven. De SQSGP vindt de manier van financiering voor nu akkoord maar we zien het graag niet meer terug in de vorm van een garantstelling voor volgprojecten. Dat daar naar andere manieren van financiering wordt gezocht. Dank u wel. Dank u wel meneer Vis. Is er nog iemand die stemming wens over dit onderwerp? Nee? Daarmee is het voorstel aangenomen. Dan gaan wij naar het integraal huishoudestingsplan onderwijs. Dan komen wij naar de partij van de arbeid meneer de Koning. Goeien dag. Dank u wel voorzitter. Bij het integraal plan huishoudestingsplan onderwijs hebben we ook gesproken over de subsidieregeling die door het rijk ter beschikking is gesteld aan het verbeteren van het binnenklimaat van de scholen. Dat is van belang voor zowel leerlingen als eerkrachten juist in deze coronatijd. De minister heeft met de bedoeling met deze regeling dat de gemeente en onderwijsbesturen gezamenlijk in overleg gaan over de financiering van een deel daarvan. De wethouder heeft toegezegd dat hij ons gaat informeren over zowel de planning van het overleg als het inhoud. De partij van de arbeidsfractie hecht er toch aan dat wij in ieder geval vinden dat de gemeente in ieder geval ook financieel een steentje moet bijdragen aan het verbeteren van het binnenklimaat en deze zondere coronatijden. Dat zal dan waarschijnlijk ook wel leiden tot als dat gebeurt tot eventuele herschikking van de Gelden. Wat ons betreft zijn we verder voor het plan maar we willen deze kanttekening in ieder geval wel gemaakt hebben. Dank u wel voorzitter. Dank u wel meneer de Koning. Is er iemand die behoefte heeft aan stemming?"
hyp = "meneer de voorzitter, omdat wat ons betreft niet rechtsgeldig is wat wij nu vaststellen. Dank u wel. Heeft u nog wel behoefte aan stemmen mevrouw van Hulst? Nou ja, wij kunnen hier dus dan niet mee instemmen, want dan stemmen we met iets in wat niet conform de regel wet en regelgeving is. Dat lijkt me niet juist meneer de voorzitter. Dank u wel, dat mag u vinden. Wij gaan de stemming openen. Stemming is geopend, gaat uw gang. 37 stemmen dames en heren. 34 voor 3 tegen. En daarmee is het voorstel aanvaard. Dan gaan wij naar agendapunt 14 coronacompensatie. Geen stemverklaringen, iemand behoefte aan stemming? Nee? dan is het voorstel bij deze aangenomen. Dan gaan wij naar agendpunt 15 en dat is, ho! Er glipte een papiertje uit mijn vingers. Het ambachtcentrum Samen Circulair. Ik zie even kijken, insprekers, tenminste, insprekers, een stemverklaring van CDA, conservatief Hoekswaard en de SGP en we beginnen bij conservatief Hoekswaard. Meneer de Jong. Dank u wel, voorzitter. We hebben 3 zorgpunten die ik in deze stemverklaring noem. onze zorg hebben wij over het feit dat de motie nog niet is uitgevoerd en in dit kader voor een mogelijk ongewenste onduidelijkheid kan zorgen. Het tweede punt is het samenwerking met de overbrugging omdat het gaat om zeer kwetsbare mensen. Wij hopen dat de gemeente hier voldoende aandacht aan blijft schenken is dat de wethouder. Wij vinden het opmerkelijk dat de wethouder geen antwoord kan geven op de vraag of dit project zou stoppen als er een renteloos lening gegeven zou worden. In plaats van een aanwezigheid voor de aanwezigheid  voor de Het is een project waar veel overheidssubsidie naartoe gaat en wat zwaar steunt op zorgbudgetten. Er werken kwetsbare mensen die geen enkele vorm van garantie hebben op een gelijkwaardig werktraject als dit plan mocht mislukken. Ook is er geen concreet plan om kringloopwinkels en repaircaf\u00e9s in andere dorpen te faciliteren of aan te laten haken bij het initiatief. De positionering en de bijdrage aan circulaire economie blijft voor het CDA onduidelijk. Wij vinden dat het college ervoor had moeten zorgen dat wij als raad een goede business case voorgelegd hadden gekregen. Toch zullen wij vanavond voor de garantstelling stemmen omdat wij het idee van een pilot voor circulaire economie wel zien zitten. Wij verwachten dat van het college dat zij een vinger aan de pols houdt of er goede stappen worden genomen om business partners aan te haken en fondswerving rond te maken. Dank u wel voorzitter. Dank u wel mevrouw Voorthoerden tot slot de SGP. Dank u wel voorzitter. De SGP kan instellen met dit voorstel omdat het naast het duurzaamheidsaspect ook het verbinden van het dorpskernens centraal staat en een mogelijkheid biedt voor werkervaring voor mensen met een afstand tot de arbeidsmarkt. Wel hebben we onze zorgen of willen we onze zorgen uit over de waarboring van de continu\u00efteit We zien graag dat er toch een concrete inspanningsverplichting komt als die garantstelling wordt gegeven. De SQSGP vindt de manier van financiering voor nu akkoord maar we zien het graag niet meer terug in de vorm van een garantstelling voor volgprojecten. Dat daar naar andere manieren van financiering wordt gezocht. Dank u wel. Dank u wel meneer Vis. Is er nog iemand die stemming wens over dit onderwerp? Nee? Daarmee is het voorstel aangenomen. Dan gaan wij naar het integraal huishoudestingsplan onderwijs. Dan komen wij naar de partij van de arbeid meneer de Koning. Goeien dag. Dank u wel voorzitter. Bij het integraal plan huishoudestingsplan onderwijs hebben we ook gesproken over de subsidieregeling die door het rijk ter beschikking is gesteld aan het verbeteren van het binnenklimaat van de scholen. Dat is van belang voor zowel leerlingen als eerkrachten juist in deze coronatijd. De minister heeft met de bedoeling met deze regeling dat de gemeente en onderwijsbesturen gezamenlijk in overleg gaan over de financiering van een deel daarvan. De wethouder heeft toegezegd dat hij ons gaat informeren over zowel de planning van het overleg als het inhoud. De partij van de arbeidsfractie hecht er toch aan dat wij in ieder geval vinden dat de gemeente in ieder geval ook financieel een steentje moet bijdragen aan het verbeteren van het binnenklimaat en deze zondere coronatijden. Dat zal dan waarschijnlijk ook wel leiden tot als dat gebeurt tot eventuele herschikking van de Gelden. Wat ons betreft zijn we verder voor het plan maar we willen deze kanttekening in ieder geval wel gemaakt hebben. Dank u wel voorzitter. Dank u wel meneer de Koning. Is er iemand die behoefte heeft aan stemming?"
error_rate = wer(ref, hyp)
wer_rates_covid.append(error_rate)
print(f"WER: {error_rate:.2f}")

# Hoekschewaard - 2021 - 833622.mp4 - 23:35
ref = "Totaal in de verschillende tranches, in de verschillende coronapakketten 0,8 miljoen. En dat maakt onderdeel uit van deze budgetoverheveling. Dan is in het raadvoorstel over de slotrapportage is een voorstel aan u opgenomen om 1,2 miljoen euro beschikbaar te stellen voor de invoering van de Omgeemingswet. En daar zal mijn collega Mark Dallem u straks een nadere toelichting opgeven. Dan kom ik bij het vervolgproces. De slotrapportage zit dus vandaag in de beeldvormende vergadering, volgende week in de oordeelsvormende vergadering en uiteindelijk de besluitvormende vergadering in december. Dan zijn we nu toegekomen aan de Omgeemingswet. Tenzij er nog vragen zijn over de slotrapportage? Nee, ik zie geen handen. Oh ja, toch wel. Heer van der Linden. En heer Rajen. En heer van der Linden, aan uw woord. Ja, dank u wel. Vraag, bij de begroting hebben we gesproken over een aantal risico's die als PM-posten zijn benoemd. Hoe komen die terug in dit overzicht? Is daar al iets meer over te duiden op dit moment? Of zit die daar even werk? Of niet? Wint Loeff? Ja, een terechte vraag. Dat zijn de risico's die wij hebben opgenomen in de begroting en ook bij het raadvoorstel van de begroting, waaronder bijvoorbeeld de herverdeling van het gemeentenfonds. Die risico's hebben effecten op de jaarschijven 20, 22 en verder. Dus die hebben echt betrekking op de begroting en de jaren daarna. En de risico's die wij hier zien, die hebben we apart in de slotrapportage opgenomen en die hebben betrekking op het saldo van 20, 21. Dus deze risico's in de paragraaf aandachtspunten en risico's in de slotrapportage hebben betrekking op 21. En de PM-posten waar u op doelt hebben betrekking op 22 en de jaren daarna. Dus die zullen terugkomen ook in de zomerapportage van volgend jaar. Ja, dat begrijp ik. Maar is er op dit moment al iets meer over te zeggen? Dat is mijn vraag. Nee, daar is nog niks meer over te zeggen. Daar gaat mijn naam over de herverdeling van het gemeentenfonds. Nou, dat ligt voor. Er is een advies van de ROB geweest. Raad van Openbaar Bestuur. Daarvan heeft de VNG gevraagd aan de minister om daarop te reageren. En er worden vanuit diverse gemeentes ook acties gevoerd om die herverdeling zeg maar tegen te gaan en met name vanuit de nadeelgemeentes. Begrijpelijk. Dus daar is op dit moment nog niks meer over te vertellen. Andere risico's die daar waren opgenomen waren de effecten van de omgevingswet. Ook daarvan hebben we op dit moment nog geen extra beeld. Maar zodra wij die effecten wel weten, dan komen we daar bij u op terug. Is dat voldoende antwoord op uw vraag, meneer van der Linde? Jazeker, dank u wel. Ok\u00e9, dan de heer Rijer aan u het woord. Ja, dank voorzitter. Nou, heel mooi om te zien dat die cultuurgelden zo mooi mee verhuizen naar de begroting van volgend jaar. Ik had eigenlijk een gedachte bij het geld dat we op opleidingen voor het ambtelijk apparaat. Dat is natuurlijk niet aan ons, maar ik vroeg me wel af zou het niet verstandig zijn om ook die gelden gewoon beschikbaar te houden. Want ik kan me wel voorstellen dat je dat ook nog wil inhalen. Is dat een hele rare opmerking? Heer Loef. Ja, ook terechte opmerking hoor. De opleidingen die in principe is daar voor volgend jaar weer opnieuw gewoon zijn daar middelen voor beschikbaar. En we hebben nu dit jaar gezien dat we vanwege coronageld hebben overgehouden. Dat was het jaar daarvoor ook. Dus de verwachting is dat we met de middelen die we voor volgend jaar hebben, dat is 2 procent van de loonsom, dat we daar in principe gewoon de opleidingen mee kunnen bekostigen. Dus dat er voldoende middelen zijn om de opleidingen gewoon voor volgend jaar te kunnen betalen. Dus daar hoeven geen extra middelen te worden overgeheveld. Ok\u00e9, nou dank voor het antwoord. Ik zal u nu niet vermoeien met mijn gedachte daarover. Het is wel goed om zo met te praten. Dank u wel. Een vooraankondiging voor volgende week wellicht. Dan zie ik verder geen vragen meer. Dan is de toelichting en de beantwoording helder geweest. En ik mocht het woord geven aan de heer Ridderhof volgens mij. Mark Dalm. Mijn collega Mark Dalm. Die zal de toelichting op de omgevingswet geven. Ik zal gewoon de presentatie laten staan. Prima. Meneer Dalm, al jullie het woord. Prima voorzitter, ik zie alleen nog een vraag in beeld komen. Meneer Lijand, heeft u nog een vraag? Klopt, ja. Die herverdeling van die rijksmiddelen neemt het missionaire kabinet het nog voor zijn rekening is de vraag. Dat heeft me al ingrijpend besluit."
hyp = "Totaal in de verschillende tranches, in de verschillende coronapakketten 0,8 miljoen. En dat maakt onderdeel uit van deze budgetoveredeling. Dan is in het raadvoorstel oh de slotrapportage is een voorstel aan u opgenomen om 1,2 miljoen euro beschikbaar te stellen voor de invoering van de omgevingswet en daar zal mijn collega Mark Dallem u straks een nadere toelichting op geven. Dan kom ik bij het vervolgproces. Nou de slotrapportage zit dus vandaag in de beeldvormende vergadering, volgende week in de oordeelsvormende vergadering en uiteindelijk de besluitvormende vergadering in december. Dan zijn we nu toegekomen aan de omgevingswet, tenzij er nog vragen zijn over de slotrapportage? Nee, ik zie geen handen. Oh jawel, toch wel. De heer van der Linden. En heer Raaijer. De heer van der Linden, aan uw woord. Ja, dank u wel. Een vraag, sorry, bij de begroting hebben we gesproken over een aantal risico's die als PM-posten zijn benoemd. Hoe komen die terug in dit overzicht? Is daar al iets meer over te duiden op dit moment? Of zitten die daar in verwerk? Of niet? Wint Loeff? Ja, nee, een terechte vraag. Dat zijn de risico's die wij hebben opgenomen in de begroting en ook bij het raadvoorstel van de begroting, waaronder bijvoorbeeld de herverdeling van het gemeentenfonds. Die risico's hebben effecten op de jaarschijven 2022 en verder. Dus die hebben echt betrekking op de begroting en de jaren daarna. En de risico's die wij hier zien, die hebben we apart in de slotrapportage opgenomen en die hebben betrekking op het saldo van 2021, dus deze risico's in de paragraaf aandachtspunten en risico's in de slotrapportage hebben betrekking op 21 en de PM-posten waar u op doelt hebben betrekking op 22 en de jaren daarna. Dus die zullen terugkomen ook in de zomerapportage van volgend jaar. Ja, dat begrijp ik. Maar is er op dit moment al iets meer over te zeggen, dat is mijn vraag. Nee, daar is nog niks meer over te zeggen. Daar gaat mijn naam over de herverdeling van het gemeentenfonds. Nou, dat ligt voor. Er is een advies van de ROB geweest. Raad van Openbaar Bestuur. Daarvan heeft de VNG gevraagd aan de minister om daarop te reageren. En er worden vanuit diverse gemeentes ook acties uitgevoerd om die herverdeling nou ja zeg maar, tegen te gaan en met name vanuit de nadeelgemeentes. Begrijpelijk. Dus daar is op dit moment nog niks meer over te vertellen. Andere risico's die waren, daar waren opgenomen, waren de effecten van de omgevingswet. Ook daarvan hebben we op dit moment nog geen extra beeld. Maar zodra wij die effecten wel weten, dan komen we daar bij u op terug. Is dat voldoende antwoord op uw vraag, meneer van der Linden? Jazeker, dank u wel. Oke, dan de heer Raaijer, aan u het woord. Ja, dank voorzitter. uhm, nou, heel mooi om te zien dat die cultuur gelden zo mooi mee verhuizen naar de begroting van volgend jaar. Ik had eigenlijk een soort gelijke gedachte bij het geld dat we overhouden op opleidingen voor het ambtelijk apparaat. Dat is natuurlijk niet aan ons, ik vroeg me wel af zou het niet verstandig zijn om ook die gelden gewoon beschikbaar te houden want ik kan me wel voorstellen dat je dat ook nog wil inhalen. Of is dat een hele rare opmerking? De heer Loef. Ja, ook een terechte opmerking hoor. De opleidingen die in principe is daar voor volgend jaar weer opnieuw gewoon zijn daar middelen voor beschikbaar. En we hebben nu dit jaar gezien dat we vanwege coronageld hebben overgehouden. Dat was het jaar daarvoor. Dus de verwachting is dat we met de middelen die we volgend jaar hebben, dat is 2\% van de loon som, dat we daar in principe gewoon de opleidingen mee kunnen, mee kunnen bekostigen. Dus dat er voldoende middelen zijn om de opleidingen gewoon voor volgend jaar te kunnen betalen. Dus daar hoeven geen extra te worden overgeheveld. Oke, nou dank voor het antwoord. Ik zal u nu niet vermoeien met mijn gedachte daarover. Dat is wel goed om er zo over te praten denk ik. Dank u wel. Nou een vooraankondiging voor volgende wellicht. Dan zie ik verder geen vragen meer. Dan is de toelichting en de beantwoording helder geweest. En ik mocht het woord geven aan de heer Ridderhof volgens mij he? Mark Dalm. oh mark dalm. Mijn collega Mark Dalm. Die zal de toelichting op de omgevingswet geven. Ik zal gewoon de de presentatie laten staan. Prima. Meneer Dalm, aan jullie het woord. Prima voorzitter, ik zie alleen nog een vraag in beeld komen. Oh? Ja. Meneer Lieand, heeft u nog een vraag? Klopt, ja. Die herverdeling van van die rijksmiddelen neemt het de missionaire kabinet het nog voor zijn rekening is de vraag. Dat heeft me al ingrijpend besluit. Ik denk het niet inderdaad."
error_rate = wer(ref, hyp)
wer_rates_covid.append(error_rate)
print(f"WER: {error_rate:.2f}")

In [ ]:
def get_info(data):
    mean = round(statistics.mean(data), 2)
    stdev = round(statistics.stdev(data), 2)
    variance = round(statistics.variance(data), 2)
    median = round(statistics.median(data), 2)
    max_e = round(max(data), 2)
    min_e = round(min(data), 2)

    return mean, stdev, variance, median, max_e, min_e

In [ ]:
mean, stdev, variance, median, max_e, min_e = get_info(wer_rates)
print(f"Whisper average WER: {mean}")
print(f"Whisper WER stdev: {stdev}")
print(f"Whisper WER variance: {variance}")
print(f"Whisper WER median: {median}")
print(f"Whisper WER max: {max_e}")
print(f"Whisper WER min: {min_e}")

print()
mean, stdev, variance, median, max_e, min_e = get_info(wer_rates_covid)
print(f"Covid Whisper average WER: {mean}")
print(f"Covid Whisper WER stdev: {stdev}")
print(f"Covid Whisper WER variance: {variance}")
print(f"Covid Whisper WER median: {median}")
print(f"Covid Whisper WER max: {max_e}")
print(f"Covid Whisper WER min: {min_e}")

print()
wer_rates_all = wer_rates + wer_rates_covid
mean, stdev, variance, median, max_e, min_e = get_info(wer_rates_all)
print(f"Total Whisper average WER: {mean}")
print(f"Total Whisper WER stdev: {stdev}")
print(f"Total Whisper WER variance: {variance}")
print(f"Total Whisper WER median: {median}")
print(f"Total Whisper WER max: {max_e}")
print(f"Total Whisper WER min: {min_e}")

In [ ]:
START = 3
DUR = 4
SPEAKER = 7


def round_to_nearest_interval(value, interval=0.125):
    return round(value / interval) * interval


def get_ref(filepath, start, end):
    reference = Annotation()
    with open(filepath, "r") as f:
        for line in f:
            info = line.strip().replace("\n", "").split()
            start_segm = float(info[START])
            end_segm = float(info[START]) + float(info[DUR])
            if start_segm >= start and end_segm <= end:
                reference[
                    Segment(
                        (start_segm),
                        (end_segm),
                    )
                ] = info[SPEAKER]
    # print(reference)
    # print("----")
    return reference


def get_hyp(filepath, start, end):
    data_path = "diar_data.json"
    with open(data_path, "r") as f:
        data = json.load(f)

    hypothesis = Annotation()
    for entry in data:
        if entry["file"] == filepath:
            for e in entry["data"]:
                start_segm = e[0]
                end_segm = e[1]
                if start_segm >= start and end_segm <= end:
                    hypothesis[
                        Segment(
                            (start_segm), (end_segm)
                        )
                    ] = e[2]
            return hypothesis

    print(f"{filepath} not found in {data_path}")

In [ ]:
def der(filepath, start, end, collar=0.0):
    diarizationErrorRate = DiarizationErrorRate(
        collar=collar,
    )
    metrics = diarizationErrorRate(
        get_ref(filepath, start, end),
        get_hyp(filepath, start, end),
        uem=Segment(start, end),
        detailed=True,
    )

    total_time = end - start
    false_alarm = metrics["false alarm"]
    missed = metrics["missed detection"]
    confusion = metrics["confusion"]
    der = metrics["diarization error rate"]

    return total_time, false_alarm, missed, confusion, der

In [ ]:
filepath = "/Volumes/Samsung_T5/data/ridderkerk/vergaderingen/2022/diorizations/938576.wav.rttm"
start = 2470
end = 2649
get_ref(filepath, start, end)


In [ ]:
get_hyp(filepath, start, end)

In [ ]:
final_data = {
    "no_collar": {"false_alarms": [], "missed": [], "confusion": [], "der": []},
    "250_collar": {"false_alarms": [], "missed": [], "confusion": [], "der": []},
    "500_collar": {"false_alarms": [], "missed": [], "confusion": [], "der": []},
}

In [ ]:
def add_to_final_data(filepath, start, end, lists):
    total_time, false_alarm, missed, confusion, der_metric = der(filepath, start, end)
    lists["no_collar"]["false_alarms"].append(false_alarm)
    lists["no_collar"]["missed"].append(missed)
    lists["no_collar"]["confusion"].append(confusion)
    lists["no_collar"]["der"].append(der_metric)

    total_time, false_alarm, missed, confusion, der_metric = der(
        filepath, start, end, collar=0.250
    )
    lists["250_collar"]["false_alarms"].append(false_alarm)
    lists["250_collar"]["missed"].append(missed)
    lists["250_collar"]["confusion"].append(confusion)
    lists["250_collar"]["der"].append(der_metric)

    total_time, false_alarm, missed, confusion, der_metric = der(
        filepath, start, end, collar=0.500
    )
    lists["500_collar"]["false_alarms"].append(false_alarm)
    lists["500_collar"]["missed"].append(missed)
    lists["500_collar"]["confusion"].append(confusion)
    lists["500_collar"]["der"].append(der_metric)

In [ ]:
filepath = "/Users/personal/Desktop/scriptie/notebooks/data/nijmegen/100000013/2022/diorizations/80ec1e6a-8c74-4f41-bcc5-6f5f27d48512.wav.rttm"
start = 3208
end = 3499
add_to_final_data(filepath, start, end, final_data)

filepath = "/Volumes/Samsung_T5/data/hoekschewaard/vergaderingen/2023/diorizations/1068492.wav.rttm"
start = 3533
end = 3798
add_to_final_data(filepath, start, end, final_data)

filepath = "/Volumes/Samsung_T5/data/ridderkerk/vergaderingen/2022/diorizations/986111.wav.rttm"
start = 3533
end = 3877
add_to_final_data(filepath, start, end, final_data)

filepath = "/Volumes/Samsung_T5/data/ridderkerk/vergaderingen/2022/diorizations/938576.wav.rttm"
start = 2320
end = 2649
add_to_final_data(filepath, start, end, final_data)

filepath = "/Volumes/Samsung_T5/data/barendrecht/vergaderingen/2024/diorizations/1203463.wav.rttm"
start = 2516
end = 2977
add_to_final_data(filepath, start, end, final_data)

In [ ]:
results = {
    collar: {
        metric: get_info(final_data[collar][metric]) for metric in final_data[collar]
    }
    for collar in final_data
}

for collar, metrics in results.items():
    for metric, values in metrics.items():
        print(f"{collar.capitalize()} {metric.replace('_', ' ')} average: {values[0]}")
        print(f"{collar.capitalize()} {metric.replace('_', ' ')} stdev: {values[1]}")
        print(f"{collar.capitalize()} {metric.replace('_', ' ')} variance: {values[2]}")
        print(f"{collar.capitalize()} {metric.replace('_', ' ')} median: {values[3]}")
        print(f"{collar.capitalize()} {metric.replace('_', ' ')} max: {values[4]}")
        print(f"{collar.capitalize()} {metric.replace('_', ' ')} min: {values[5]}")
        print()